In [9]:
import tensorflow as tf

# Percorso al tuo modello
model_path = 'modello75.keras'  # Sostituisci con il percorso effettivo

# Carica il modello
model = tf.keras.models.load_model(model_path)

In [10]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Definisci il percorso alla cartella delle immagini
test_dir = 'foto-mie'

# 2. Definisci le classi di interesse e tutte le classi del modello
classes_of_interest = ['Neutral', 'Happy', 'Surprise']
model_classes = ['Neutral', 'Happy', 'Surprise', 'Sad', 'Anger', 'Disgust', 'Fear', 'Contempt', 'Unknown']

# 3. Crea mappature tra indici e classi
model_class_to_index = {class_name: idx for idx, class_name in enumerate(model_classes)}
model_index_to_class = {idx: class_name for idx, class_name in enumerate(model_classes)}

interest_class_indices = [model_class_to_index[class_name] for class_name in classes_of_interest]

# 4. Carica le immagini e le etichette vere
image_paths = []
true_labels = []

for class_name in classes_of_interest:
    class_dir = os.path.join(test_dir, class_name)
    if not os.path.exists(class_dir):
        print(f"La cartella {class_dir} non esiste. Verifica i nomi delle cartelle.")
        continue
    for filename in os.listdir(class_dir):
        img_path = os.path.join(class_dir, filename)
        image_paths.append(img_path)
        true_labels.append(model_class_to_index[class_name])  # Etichetta come indice nel modello

# 5. Carica il modello
model_path = 'modello75.keras'  # Sostituisci con il percorso effettivo
model = tf.keras.models.load_model(model_path)

# 6. Carica e preprocessa le immagini
images = []
for img_path in image_paths:
    img = Image.open(img_path)
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=-1)
    img_array = img_array.astype('float32')
    # Se necessario, normalizza l'immagine (decommenta se il modello lo richiede)
    # img_array /= 255.0
    images.append(img_array)

images = np.array(images)
true_labels = np.array(true_labels)

# 7. Esegui le predizioni
predictions = model.predict(images)
predicted_indices = np.argmax(predictions, axis=1)

# 8. Analizza le predizioni
# Identifica dove il modello predice classi diverse da "Neutral", "Happy" o "Sadness"
incorrect_predictions = predicted_indices != true_labels

# Ottieni gli indici delle predizioni errate
incorrect_indices = np.where(incorrect_predictions)[0]

# Conta quante volte il modello ha predetto altre classi per ciascuna classe vera
from collections import Counter

# Conta le predizioni errate per ciascuna classe vera
errors_counter = Counter()

for idx in incorrect_indices:
    true_class = model_index_to_class[true_labels[idx]]
    predicted_class = model_index_to_class[predicted_indices[idx]]
    errors_counter[(true_class, predicted_class)] += 1

# 9. Stampa i risultati
total_images = len(true_labels)
total_errors = len(incorrect_indices)
accuracy = (total_images - total_errors) / total_images * 100

print(f"Totale immagini testate: {total_images}")
print(f"Totale predizioni corrette: {total_images - total_errors}")
print(f"Totale predizioni errate: {total_errors}")
print(f"Accuratezza complessiva: {accuracy:.2f}%\n")

print("Dettaglio delle predizioni errate:")
for (true_class, predicted_class), count in errors_counter.items():
    print(f"Vero: {true_class}, Predetto: {predicted_class} - {count} volte")

# 10. Matrice di confusione
# Crea una matrice di confusione includendo tutte le classi del modello
cm = confusion_matrix(true_labels, predicted_indices, labels=[model_class_to_index[c] for c in model_classes])

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=model_classes, yticklabels=model_classes, cmap='Blues')
plt.xlabel('Etichetta Predetta')
plt.ylabel('Etichetta Vera')
plt.title('Matrice di Confusione sulle Classi Neutral, Happy e Sadness')
plt.show()

# 11. Report di classificazione
report = classification_report(true_labels, predicted_indices, target_names=model_classes)
print(report)

# 12. Visualizza esempi di predizioni errate
import random

num_examples = 5  # Numero di esempi da mostrare
if total_errors > 0:
    example_indices = random.sample(list(incorrect_indices), min(num_examples, total_errors))
    for idx in example_indices:
        img = Image.open(image_paths[idx])
        true_class = model_index_to_class[true_labels[idx]]
        predicted_class = model_index_to_class[predicted_indices[idx]]
        plt.imshow(img, cmap='gray')
        plt.title(f"Vero: {true_class}, Predetto: {predicted_class}")
        plt.axis('off')
        plt.show()
else:
    print("Tutte le predizioni sono corrette.")


La cartella foto-mie\Neutral non esiste. Verifica i nomi delle cartelle.
La cartella foto-mie\Happy non esiste. Verifica i nomi delle cartelle.
La cartella foto-mie\Surprise non esiste. Verifica i nomi delle cartelle.


ValueError: math domain error

In [11]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

# 1. Definisci il percorso alla cartella delle immagini
test_dir = 'foto-mie'

# 2. Definisci le classi di interesse e tutte le classi del modello
classes_of_interest = ['Neutral', 'Happy', 'Surprise']
model_classes = ['Neutral', 'Happy', 'Surprise', 'Sad', 'Anger', 'Disgust', 'Fear', 'Contempt', 'Unknown']

# 3. Crea mappature tra indici e classi
model_class_to_index = {class_name: idx for idx, class_name in enumerate(model_classes)}
model_index_to_class = {idx: class_name for idx, class_name in enumerate(model_classes)}

interest_class_indices = [model_class_to_index[class_name] for class_name in classes_of_interest]

# 4. Carica le immagini (da una singola cartella)
image_paths = []
for filename in os.listdir(test_dir):
    img_path = os.path.join(test_dir, filename)
    image_paths.append(img_path)

# 5. Se hai delle etichette vere (manualmente fornite o lette da file)
# In questo esempio, presumo che non hai etichette vere disponibili, quindi non le utilizzeremo per il confronto
# Se hai delle etichette, puoi assegnarle qui manualmente
true_labels = None  # Nessuna etichetta disponibile, puoi impostarle se necessarie

# 6. Carica il modello
model_path = 'modello75.keras'  # Sostituisci con il percorso effettivo
model = tf.keras.models.load_model(model_path)

# 7. Carica e preprocessa le immagini
images = []
for img_path in image_paths:
    img = Image.open(img_path).convert('L')  # Converti in scala di grigi se necessario
    img = img.resize((48, 48))  # Assicurati che la dimensione corrisponda a quella del modello
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=-1)  # Aggiungi un canale per le immagini in scala di grigi
    images.append(img_array)

images = np.array(images)

# 8. Esegui le predizioni
predictions = model.predict(images)
predicted_indices = np.argmax(predictions, axis=1)

# 9. Analizza le predizioni (senza confrontare con etichette vere poiché non le abbiamo)
predicted_labels = [model_index_to_class[idx] for idx in predicted_indices]

# Stampa le predizioni per ciascuna immagine
for img_path, pred_label in zip(image_paths, predicted_labels):
    print(f"Immagine: {os.path.basename(img_path)}, Predizione: {pred_label}")

# 10. (Opzionale) Se hai le etichette vere, puoi calcolare accuratezza e generare la matrice di confusione
if true_labels is not None:
    cm = confusion_matrix(true_labels, predicted_indices, labels=[model_class_to_index[c] for c in model_classes])

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=model_classes, yticklabels=model_classes, cmap='Blues')
    plt.xlabel('Etichetta Predetta')
    plt.ylabel('Etichetta Vera')
    plt.title('Matrice di Confusione')
    plt.show()

    report = classification_report(true_labels, predicted_indices, target_names=model_classes)
    print(report)
else:
    print("Etichette vere non fornite. Solo predizioni mostrate.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
Immagine: ffhq_0.png, Predizione: Happy
Immagine: ffhq_1.png, Predizione: Happy
Immagine: ffhq_10.png, Predizione: Happy
Immagine: ffhq_11.png, Predizione: Happy
Immagine: ffhq_12.png, Predizione: Happy
Immagine: ffhq_13.png, Predizione: Happy
Immagine: ffhq_2.png, Predizione: Neutral
Immagine: ffhq_3.png, Predizione: Neutral
Immagine: ffhq_4.png, Predizione: Happy
Immagine: ffhq_5.png, Predizione: Happy
Immagine: ffhq_6.png, Predizione: Happy
Immagine: ffhq_7.png, Predizione: Neutral
Immagine: ffhq_8.png, Predizione: Happy
Immagine: ffhq_9.png, Predizione: Neutral
Etichette vere non fornite. Solo predizioni mostrate.
